#### 导入我们所需要的包

In [16]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
#warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 
tqdm.pandas()

这里我在构建训练集的时候，我把复赛训练集也拿来训练了（丰富了训练集）  
round1_iflyad_anticheat_traindata.txt为初赛的训练集   
round2_iflyad_anticheat_traindata.txt为复赛的训练集     
使用pd.read_table读取txt文件

In [ ]:
## data_concat
train1 = pd.read_table(path+"round1_iflyad_anticheat_traindata.txt")
train2 = pd.read_table(path+"round2_iflyad_anticheat_traindata.txt")
train = train1.append(train2).reset_index(drop=True)#reset_index重建索引，drop=True 是把原来索引删掉
###concat  会报错，害我浪费了一些时间
#train = pd.concat([train1,train2])
#打印训练集训练
print(train.info())

round2_iflyad_anticheat_testdata_feature_A.txt 是复赛需要预测的A榜数据集   
round2_iflyad_anticheat_testdata_feature_B.txt 是复赛需要预测的B榜数据集   

In [ ]:
test_2a = pd.read_table(path+"round2_iflyad_anticheat_testdata_feature_A.txt")
test_2b = pd.read_table(path+"round2_iflyad_anticheat_testdata_feature_B.txt")
test = test_2a.append(test_2b).reset_index(drop=True)#reset_index重建索引，drop=True 是把原来索引删掉
#test = pd.concat([test_2a,test_2b])
print(test.info())
test['label'] = -999 #这样的方式是填充test 的label 列，使得test和train对其
data = train.append(test).reset_index(drop=True)#reset_index重建索引，drop=True 是把原来索引删掉

In [ ]:
# data = pd.concat([train,test])
# 数据预处理
## 制造商预处理
#将手机厂商 make列包含某一列的的make改成统一的类型
data.loc[data['make'].str.contains('长虹', na=False), 'make'] = 'changhong'
data.loc[data['make'].str.contains('朵唯', na=False), 'make'] = 'doov'
data.loc[data['make'].str.contains('sm', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('SAMSUNG', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('三星', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('GT-', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('格力', na=False), 'make'] = 'gree'
data.loc[data['make'].str.contains('Moto G', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('Moto', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('moto', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('摩托罗拉', na=False), 'make'] = 'motorola'
data.loc[data['make'].str.contains('诺基亚', na=False), 'make'] = 'nokia'
data.loc[data['make'].str.contains('Nokia', na=False), 'make'] = 'nokia'
data.loc[data['make'].str.contains('努比亚', na=False), 'make'] = 'nubia'
data.loc[data['make'].str.contains('美图', na=False), 'make'] = 'meitu'
data.loc[data['make'].str.contains('LG-', na=False), 'make'] = 'LG'
data.loc[data['make'].str.contains('联想', na=False), 'make'] = 'lenovo'
data.loc[data['make'].str.contains('rv:', na=False), 'make'] = 'RV'
data.loc[data['make'].str.contains('rv:', na=False), 'make'] = 'RV'
data.loc[data['make'].str.contains('小辣椒', na=False), 'make'] = 'xiaolajiao'
data.loc[data['make'].str.contains('HUAWEI', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('huawei', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('荣耀', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('华为', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('-L', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('al', na=False), 'make'] = 'huawei'
data.loc[data['make'].str.contains('Blade', na=False), 'make'] = 'zte'
data.loc[data['make'].str.contains('BLADE', na=False), 'make'] = 'zte'
data.loc[data['make'].str.contains('中兴', na=False), 'make'] = 'zte'
data.loc[data['make'].str.contains('海信', na=False), 'make'] = 'hisense'
data.loc[data['make'].str.contains('Linux', na=False), 'make'] = 'Linux'
data.loc[data['make'].str.contains('乐丰', na=False), 'make'] = 'lephone'
data.loc[data['make'].str.contains('百立丰', na=False), 'make'] = 'lephone'
data.loc[data['make'].str.contains('乐视', na=False), 'make'] = 'letv'
data.loc[data['make'].str.contains('XT', na=False), 'make'] = 'Sony'
data.loc[data['make'].str.contains('htc', na=False), 'make'] = 'htc'
data.loc[data['make'].str.contains('HTC', na=False), 'make'] = 'htc'
data.loc[data['make'].str.contains('ASUS', na=False), 'make'] = 'Asus'
data.loc[data['make'].str.contains('锤子', na=False), 'make'] = 'smartisan'
data.loc[data['make'].str.contains('oppo', na=False), 'make'] = 'oppo'
data.loc[data['make'].str.contains('pb', na=False), 'make'] = 'oppo'
data.loc[data['make'].str.contains('realme', na=False), 'make'] = 'oppo'
data.loc[data['make'].str.contains('天语', na=False), 'make'] = 'k touch'
data.loc[data['make'].str.contains('Tianyu', na=False), 'make'] = 'k touch'
data.loc[data['make'].str.contains('tianyu', na=False), 'make'] = 'k touch'
data.loc[data['make'].str.contains('酷派', na=False), 'make'] = 'coolpad'
data.loc[data['make'].str.contains('索尼', na=False), 'make'] = 'sony'
data.loc[data['make'].str.contains('SONY', na=False), 'make'] = 'sony'
data.loc[data['make'].str.contains('sony', na=False), 'make'] = 'sony'
data.loc[data['make'].str.contains('小米', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('mi', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m1', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m1s', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m2', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m2s', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m2a', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m3', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('m6', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('xiaomi', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('redmi', na=False), 'make'] = 'xiaomi'
data.loc[data['make'].str.contains('魅族', na=False), 'make'] = 'meizu'
data.loc[data['make'].str.contains('360', na=False), 'make'] = '360'
data.loc[data['make'].str.contains('三星', na=False), 'make'] = 'samsung'
data.loc[data['make'].str.contains('赛博宇华', na=False), 'make'] = 'sop'
data.loc[data['make'].str.contains('金立', na=False), 'make'] = 'jinli'
data.loc[data['make'].str.contains('gionee', na=False), 'make'] = 'jinli'
data.loc[data['make'].str.contains('Gionee', na=False), 'make'] = 'jinli'
data.loc[data['make'].str.contains('vivo', na=False), 'make'] = 'vivo'
data.loc[data['make'].str.contains('VIVO', na=False), 'make'] = 'vivo'
data.loc[data['make'].str.contains('oneplus', na=False), 'make'] = 'oneplus'
data.loc[data['make'].str.contains('一加', na=False), 'make'] = 'oneplus'
## 开源预处理 型号预处理
#将一些无处理的手机型号替代成能识别的手机信号
data['model'].replace('PACM00',"OPPO R15",inplace=True)
data['model'].replace('PBAM00',"OPPO A5",inplace=True)
data['model'].replace('PBEM00',"OPPO R17",inplace=True)
data['model'].replace('PADM00',"OPPO A3",inplace=True)
data['model'].replace('PBBM00',"OPPO A7",inplace=True)
data['model'].replace('PAAM00',"OPPO R15_1",inplace=True)
data['model'].replace('PACT00',"OPPO R15_2",inplace=True)
data['model'].replace('PABT00',"OPPO A5_1",inplace=True)
data['model'].replace('PBCM10',"OPPO R15x",inplace=True)

In [ ]:
## nginxtime	bigint	请求到达服务时间，单位ms
# 通过转化成时间戳转化为天和小时的特征
# 时间特征
data['time'] = pd.to_datetime(data['nginxtime'] /1000,unit='s') + timedelta(hours=8)
data['day'] = data['time'].dt.day
data['hour'] = data['time'].dt.hour
#请求sid 里面有一个 到达的时间搓将时间搓切割出来
## 进一步研究时间
####sid 时间戳
data['sid_timestamp']=data['sid'].apply(lambda x:str(x).split('-')[-1]).astype(str)
##请求时间和到达时间相差
##判断请求时间和到达时间是否相等
data['nginxtime_diff_sid_time']=data['nginxtime'].astype('float')-data['sid_timestamp'].astype('float')
data['nginxtime_sid_timestamp'] = (data['nginxtime']==data['sid_timestamp'].astype(str)).astype(int)
## 进一步数据预处理
##将os字段变成大写
data['os'] = data[data.os.notnull()]['os'].apply(lambda x: str(x).upper())
data['os'] = data['os'].map(lambda x:str(x).upper())  

In [ ]:
# 原始开源baseline特征   
data['big_model'] = data['model'].map(lambda x: x.split(' ')[0])
data['model_equal_make'] = (data['big_model'] == data['make']).astype(int)

#制造商和机型进行交叉
data['machine'] = data['make'].astype(str) + '-' + data['model'].astype(str)

#开源特征https://blog.csdn.net/qq_41776781/article/details/97387261
import re
data['h'].replace(0.0, np.nan, inplace=True)
data['w'].replace(0.0, np.nan, inplace=True)
# all_data['ppi'].replace(0.0, np.nan, inplace=True)
# cols = ['h', 'w', 'ppi']
cols = ['h', 'w']
gp_col = 'make'
for col in tqdm(cols):
    na_series = data[col].isna()
    names = list(data.loc[na_series, gp_col])
    # 使用均值 或者众数进行填充缺失值
    # df_fill = all_data.groupby(gp_col)[col].mean()
    df_fill = data.groupby(gp_col)[col].agg(lambda x: stats.mode(x)[0][0])
    t = df_fill.loc[names]
    t.index = data.loc[na_series, col].index
    # 相同的index进行赋值
    data.loc[na_series, col] = t
    
# 开源iphttps://blog.csdn.net/qq_41776781/article/details/97387261
data['ip0'] = data['ip'].apply(lambda x:'.'.join(str(x).split('.')[:1]))
data['ip1'] = data['ip'].apply(lambda x:'.'.join(str(x).split('.')[0:2]))
data['ip2'] = data['ip'].apply(lambda x:'.'.join(str(x).split('.')[0:3]))

data['reqrealip0'] = data['reqrealip'].apply(lambda x:'.'.join(str(x).split('.')[:1]))
data['reqrealip1'] = data['reqrealip'].apply(lambda x:'.'.join(str(x).split('.')[0:2]))
data['reqrealip2'] = data['reqrealip'].apply(lambda x:'.'.join(str(x).split('.')[0:3]))

#判断二者是否相等
data['ip_reqrealip']   = (data['ip'] == data['reqrealip']).astype(int)
data['adidmd5_openudidmd5_d']=(data['adidmd5']==data['openudidmd5']).astype(int)
##判断md5是否相等
# data['openudidmd5_adidmd5']=(data['openudidmd5']==data['adidmd5']).astype(int) 
#下面暴力好像掉分
# data['ip_equal_1'] = (data['ip1'] == data['reqrealip1']).astype(int)
# data['ip_equal_2'] = (data['ip2'] == data['reqrealip2']).astype(int)
#继续深挖 osv
data['osv0'] = data['osv'].astype(str).map(lambda x:'.'.join(x.split('.')[:1]))
data['osv1'] = data['osv'].astype(str).map(lambda x:'.'.join(x.split('.')[0:2]))
data['osv2'] = data['osv'].astype(str).map(lambda x:'.'.join(x.split('.')[0:3]))
## 判断ip 和reqrealip 是否相等判断欺诈

#既然特征在catboost上面表现优越，在原来类别特征之间找新的交叉特征
### ip 暴力与设备进行暴力交叉
## 找城市，省份欺诈设备族群,ip 和 抬头ip过于稀疏，所以只拿province 和city 和设备
###['lan','make','model','osv','dvctype','imeimd5','os']暂时就这些，其他的nunique 过于少
#剔除重要性差特征
province_city = ['province','city']
pc_cos = ['make','model','osv','dvctype','imeimd5','lan','macmd5']#os,
for i in province_city:
    for j in pc_cos:
        data['twoPC_'+str(i)+'_'+str(j)] = data[i].astype(str)+'_'+data[j].astype(str)
#媒体暴力交互
##媒体信息和设备交互欺诈族群寻找
###媒体信息有 ['pkgname','ver','adunitshowid','mediashowid'，'apptype']
###设备信息有['adidmd5','imeimd5','idfamd5','openudidmd5','macmd5','model','make','carrier','osv','lan']剔除了仅仅只有os
### 将媒体信息与全部设备信息进行暴力交叉
Media_Information = ['pkgname','ver','adunitshowid','mediashowid','apptype']#按顺序开始取字段
Device_Information = ['adidmd5','imeimd5','openudidmd5','macmd5','model','make','carrier','osv','lan']#,'idfamd5'
for ii in Media_Information:
    for jj in Device_Information:
        data['twoMD_'+str(ii)+'_'+str(jj)] = data[ii].astype(str)+'_'+data[jj].astype(str)
#ip 和默认下载匹配app交互暴力交互ip
Ip_Information=['ip','reqrealip']
Id_Information=['adunitshowid']#'mediashowid','mediashowid',好像过拟合
for ii in Ip_Information:
    for jj in Id_Information:
        data['twoMD_'+str(ii)+'_'+str(jj)] = data[ii].astype(str)+'_'+data[jj].astype(str)
## 强特'imeimd5'暴力交叉
##设备信息自身类别交互。先选出设备唯一id，一般设备id为International Mobile Equipment Identity IMEI识别码。
Za_Information = ['model','osv','ver','dvctype','carrier']#按顺序开始取字段
Imd_Information = ['imeimd5']#,'idfamd5',其他ID好像过拟合了
for ii in Za_Information:
    for jj in Imd_Information:
        data['twoMD_'+str(ii)+'_'+str(jj)] = data[ii].astype(str)+'_'+data[jj].astype(str)


##设备信息自身类别交互。通过操作系统和本身设备交互
### 看看ime5和其他设备和部分设备是如何交互
Device_Information3=['model','make','ntt']
for iiii in Device_Information3:
    data['twoMD_'+str('osv')+'_'+str(iiii)] = data['osv'].astype(str)+'_'+data[iiii].astype(str)

In [ ]:
data['size'] = (np.sqrt(data['h'] ** 2 + data['w'] ** 2) / 2.54) / 1000
data['ratio'] = data['h'] / data['w']
data['px'] = data['ppi'] * data['size']
data['mj'] = data['h'] * data['w']

###查看该用户id用过多少个机型和设备等等(人与人的交互)开源特征：
sid_cro = ['ver','apptype','ntt','h','w','ppi','ratio','city','dvctype']
for x in sid_cro:
    data['sid'+'_'+'use'+'_'+str(x)+'_count'] = data.groupby([x])['sid'].transform('count')

In [ ]:
feature_name = [i for i in data.columns if i not in ['sid', 'label', 'time', 'day']]
cat_list = [i for i in cat_col if i not in ['nginxtime']]

feature_importances = pd.DataFrame()
feature_importances['feature'] = feature_name
all_zero_feature=[]
X_train = data[data['label']!=-999]
y = data[data['label']!=-999]['label']
X_test = data[data['label']==-999]
print(X_train.shape,X_test.shape)
oof = np.zeros(X_train.shape[0])
prediction = np.zeros(X_test.shape[0])
seeds = [2018, 2019,4096, 2048, 1024]
num_model_seed = 1
for model_seed in range(num_model_seed):
    oof_cat = np.zeros(X_train.shape[0])
    prediction_cat=np.zeros(X_test.shape[0])
    skf = StratifiedKFold(n_splits=7, random_state=seeds[model_seed], shuffle=True)
    for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
        print(index)
        train_x, test_x, train_y, test_y = X_train[feature_name].iloc[train_index], X_train[feature_name].iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        print(train_x.shape)
        print(train_y.shape)
        print(test_x.shape)
        print(test_y.shape)
        cbt_model = cbt.CatBoostClassifier(iterations=5000,learning_rate=0.1,max_depth=7,
                                           l2_leaf_reg=1,verbose=100,early_stopping_rounds=500,task_type='GPU',eval_metric='F1',cat_features=cat_list)
        cbt_model.fit(train_x[feature_name], train_y,eval_set=(test_x[feature_name],test_y))            
        oof_cat[test_index] += cbt_model.predict_proba(test_x)[:,1]
        prediction_cat += cbt_model.predict_proba(X_test[feature_name])[:,1]/7       
        featureimportance=cbt_model.get_feature_importance(prettified=True)   
        importance_0=list(featureimportance[featureimportance['Importances']==0]['Feature Index'])
        all_zero_feature.extend(importance_0)
        feature_importances['fold-{}'.format(index+1)] = cbt_model.get_feature_importance()
        featureimportance.to_csv(path+'feats{}.csv'.format(index),index=False)        
    print('F1',f1_score(y, np.round(oof_cat)))    
    oof += oof_cat / num_model_seed
    prediction += prediction_cat / num_model_seed
print('score',f1_score(y, np.round(oof))) 
# write to csv
submit = test[['sid']]
submit['label'] =prediction
submit.to_csv(path+"tmp/bs_gl1.csv",index=False)
submit['label'] = (prediction>=0.5).astype(int)
print(submit['label'].value_counts())
submit.to_csv(path+"tmp/dt1.csv",index=False)